# The Opportunity Within Amtrak - Data Cleaning Notebook

This project focuses on drilling down to a focused audience for Amtrak to target with climate-focused messaging. In order to find such a specific audience, data from multiple sources needs to be collected, cleaned, and joined together. This notebook handles climate opinion, census, and college data prior to mapping a final recommendation. 

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")



## Climate Opinion Data

This data from a Yale study on national Climate Opinions gives scores by metro area (CBSA) for certain questions about climate change. The dataset includes the percentage of people who answered yes and no to each question. 

In [2]:
opinions = pd.read_csv('Yale_Climate_Opinions.csv')
#only take the opinions by metro area, not state or other geography
cbsa_opinions = opinions[opinions.GeoType=="CBSA"]
pd.options.display.max_columns=100

In [3]:
#we are only going to focus on a few relevant questions, dropping everything else
cbsa_opinions.drop(labels=['affectweatherOppose', 'affectweather', 'harmplantsOppose', 
                          'harmplants', 'devharmOppose','devharm','harmUSOppose', 'harmUS',
                          'personalOppose', 'personal', 'teachGWOppose', 'teachGW', 'gwvoteimpOppose',
                          'gwvoteimp', 'mediaweeklyOppose', 'mediaweekly','rebatesOppose','rebates',
                          'drillANWROppose', 'drillANWR', 'drilloffshoreOppose', 'drilloffshore', 'CO2limitsOppose',
                          'CO2limits', 'corporationsOppose','corporations', 'presidentOppose','president',
                          'congressOppose','congress', 'governorOppose','governor','localofficialsOppose',
                          'localofficials', 'reducetaxOppose','reducetax', 'timingOppose',
                          'timing', 'futuregenOppose','futuregen', 'consensusOppose', 'consensus',
                          'priorityOppose', 'priority', 'fundrenewables', 'fundrenewablesOppose',
                          'supportRPS', 'supportRPSOppose'], axis=1, inplace=True)

In [4]:
cbsa_opinions.head(3)

,GeoType,GEOID,GeoName,TotalPop,discuss,discussOppose,citizens,citizensOppose,regulate,regulateOppose,happening,happeningOppose,human,humanOppose,worried,worriedOppose
3630,CBSA,10100,"Aberdeen, SD",32537,32.062,67.913,59.163,12.432,70.796,28.165,66.159,15.979,51.649,37.041,56.314,43.735
3631,CBSA,10140,"Aberdeen, WA",57009,36.515,63.218,61.745,11.793,71.429,27.738,69.182,14.057,53.290,34.998,58.796,41.183
3632,CBSA,10180,"Abilene, TX",129610,29.539,70.491,56.557,15.176,66.797,33.178,60.471,18.649,49.543,38.356,54.973,45.151


In [5]:
#the following are the national averages for each question according to the 
#Yale study website
discuss_average= 35
regulate_average = 75
citizens_average = 64
happening_average = 72
human_average = 57
worried_average = 63

av_list = [discuss_average, regulate_average, citizens_average, happening_average, human_average,worried_average]
col_list = ['discuss', 'regulate', 'citizens', 'happening', 'human', 'worried']

In [6]:
#measure each metro area's distance (positive or negative) from the 
#national average on each question
def distance_from_nat_average(row, nat_average):
    difference = row-nat_average 
    return difference

for av, col in zip(av_list, col_list):
    cbsa_opinions[f'{col}_distance_from_natav'] = cbsa_opinions.apply(lambda x: distance_from_nat_average(x[col], av), axis=1)


In [7]:
cbsa_opinions.head(3)

,GeoType,GEOID,GeoName,TotalPop,discuss,discussOppose,citizens,citizensOppose,regulate,regulateOppose,happening,happeningOppose,human,humanOppose,worried,worriedOppose,discuss_distance_from_natav,regulate_distance_from_natav,citizens_distance_from_natav,happening_distance_from_natav,human_distance_from_natav,worried_distance_from_natav
3630,CBSA,10100,"Aberdeen, SD",32537,32.062,67.913,59.163,12.432,70.796,28.165,66.159,15.979,51.649,37.041,56.314,43.735,-2.938,-4.204,-4.837,-5.841,-5.351,-6.686
3631,CBSA,10140,"Aberdeen, WA",57009,36.515,63.218,61.745,11.793,71.429,27.738,69.182,14.057,53.290,34.998,58.796,41.183,1.515,-3.571,-2.255,-2.818,-3.710,-4.204
3632,CBSA,10180,"Abilene, TX",129610,29.539,70.491,56.557,15.176,66.797,33.178,60.471,18.649,49.543,38.356,54.973,45.151,-5.461,-8.203,-7.443,-11.529,-7.457,-8.027


In [8]:
cbsa_opinions.drop(labels=['discussOppose', 'citizensOppose', 'regulateOppose', 'happeningOppose', 'humanOppose',
                          'worriedOppose'], axis=1, inplace=True)
cbsa_opinions.reset_index(inplace=True)

In [9]:
#sum up all of the distances from the national average to get an 
#overall sense of how that metro area compares to the rest of the country 
#on climate
cbsa_opinions['sum_of_distances'] = cbsa_opinions['worried_distance_from_natav'] + cbsa_opinions['happening_distance_from_natav']+cbsa_opinions['citizens_distance_from_natav']

In [ ]:
#save the cleaned data
cbsa_opinions.to_csv('Cleaned_Climate_Opinions.csv')

## Census Data

Age could also be an interesting factor to take into account when choosing a target audience. Below we clean census data for each metro area which summarizes the age statistics for the area. 

In [10]:
census_age = pd.read_csv('Census_Age_Data.csv')
census_age.head(3)

,GEO_ID,NAME,S0101_C01_001E,S0101_C01_001M,S0101_C01_002E,S0101_C01_002M,S0101_C01_003E,S0101_C01_003M,S0101_C01_004E,S0101_C01_004M,S0101_C01_005E,S0101_C01_005M,S0101_C01_006E,S0101_C01_006M,S0101_C01_007E,S0101_C01_007M,S0101_C01_008E,S0101_C01_008M,S0101_C01_009E,S0101_C01_009M,S0101_C01_010E,S0101_C01_010M,S0101_C01_011E,S0101_C01_011M,S0101_C01_012E,S0101_C01_012M,S0101_C01_013E,S0101_C01_013M,S0101_C01_014E,S0101_C01_014M,S0101_C01_015E,S0101_C01_015M,S0101_C01_016E,S0101_C01_016M,S0101_C01_017E,S0101_C01_017M,S0101_C01_018E,S0101_C01_018M,S0101_C01_019E,S0101_C01_019M,S0101_C01_020E,S0101_C01_020M,S0101_C01_021E,S0101_C01_021M,S0101_C01_022E,S0101_C01_022M,S0101_C01_023E,S0101_C01_023M,S0101_C01_024E,S0101_C01_024M,...,S0101_C06_014E,S0101_C06_014M,S0101_C06_015E,S0101_C06_015M,S0101_C06_016E,S0101_C06_016M,S0101_C06_017E,S0101_C06_017M,S0101_C06_018E,S0101_C06_018M,S0101_C06_019E,S0101_C06_019M,S0101_C06_020E,S0101_C06_020M,S0101_C06_021E,S0101_C06_021M,S0101_C06_022E,S0101_C06_022M,S0101_C06_023E,S0101_C06_023M,S0101_C06_024E,S0101_C06_024M,S0101_C06_025E,S0101_C06_025M,S0101_C06_026E,S0101_C06_026M,S0101_C06_027E,S0101_C06_027M,S0101_C06_028E,S0101_C06_028M,S0101_C06_029E,S0101_C06_029M,S0101_C06_030E,S0101_C06_030M,S0101_C06_031E,S0101_C06_031M,S0101_C06_032E,S0101_C06_032M,S0101_C06_033E,S0101_C06_033M,S0101_C06_034E,S0101_C06_034M,S0101_C06_035E,S0101_C06_035M,S0101_C06_036E,S0101_C06_036M,S0101_C06_037E,S0101_C06_037M,S0101_C06_038E,S0101_C06_038M
0,id,Geographic Area Name,Estimate!!Total!!Total population,Margin of Error!!Total!!Total population,Estimate!!Total!!Total population!!AGE!!Under ...,Margin of Error!!Total!!Total population!!AGE!...,Estimate!!Total!!Total population!!AGE!!5 to 9...,Margin of Error!!Total!!Total population!!AGE!...,Estimate!!Total!!Total population!!AGE!!10 to ...,Margin of Error!!Total!!Total population!!AGE!...,Estimate!!Total!!Total population!!AGE!!15 to ...,Margin of Error!!Total!!Total population!!AGE!...,Estimate!!Total!!Total population!!AGE!!20 to ...,Margin of Error!!Total!!Total population!!AGE!...,Estimate!!Total!!Total population!!AGE!!25 to ...,Margin of Error!!Total!!Total population!!AGE!...,Estimate!!Total!!Total population!!AGE!!30 to ...,Margin of Error!!Total!!Total population!!AGE!...,Estimate!!Total!!Total population!!AGE!!35 to ...,Margin of Error!!Total!!Total population!!AGE!...,Estimate!!Total!!Total population!!AGE!!40 to ...,Margin of Error!!Total!!Total population!!AGE!...,Estimate!!Total!!Total population!!AGE!!45 to ...,Margin of Error!!Total!!Total population!!AGE!...,Estimate!!Total!!Total population!!AGE!!50 to ...,Margin of Error!!Total!!Total population!!AGE!...,Estimate!!Total!!Total population!!AGE!!55 to ...,Margin of Error!!Total!!Total population!!AGE!...,Estimate!!Total!!Total population!!AGE!!60 to ...,Margin of Error!!Total!!Total population!!AGE!...,Estimate!!Total!!Total population!!AGE!!65 to ...,Margin of Error!!Total!!Total population!!AGE!...,Estimate!!Total!!Total population!!AGE!!70 to ...,Margin of Error!!Total!!Total population!!AGE!...,Estimate!!Total!!Total population!!AGE!!75 to ...,Margin of Error!!Total!!Total population!!AGE!...,Estimate!!Total!!Total population!!AGE!!80 to ...,Margin of Error!!Total!!Total population!!AGE!...,Estimate!!Total!!Total population!!AGE!!85 yea...,Margin of Error!!Total!!Total population!!AGE!...,Estimate!!Total!!Total population!!SELECTED AG...,Margin of Error!!Total!!Total population!!SELE...,Estimate!!Total!!Total population!!SELECTED AG...,Margin of Error!!Total!!Total population!!SELE...,Estimate!!Total!!Total population!!SELECTED AG...,Margin of Error!!Total!!Total population!!SELE...,Estimate!!Total!!Total population!!SELECTED AG...,Margin of Error!!Total!!Total population!!SELE...,Estimate!!Total!!Total population!!SELECTED AG...,Margin of Error!!Total!!Total population!!SELE...,...,Estimate!!Percent Female!!Total population!!AG...,Margin of Error!!Percent Female!!Total populat...,Estimate

In [11]:
#take out some unneeded characters from metro names
def clean_name(row):
    if 'Micro Area' in row:
        new_name = row.replace(' Micro Area', "")
    elif 'Metro Area' in row: 
        new_name = row.replace(' Metro Area', "")
    else: 
        new_name=row
    return new_name



In [12]:
#shift the rows so that row 1 becomes the column names and
#isn't included in the data itself
pd.options.display.max_columns=200
census_age.columns = census_age.iloc[0]
census_age = census_age[1:]

census_age.head(3)


,id,Geographic Area Name,Estimate!!Total!!Total population,Margin of Error!!Total!!Total population,Estimate!!Total!!Total population!!AGE!!Under 5 years,Margin of Error!!Total!!Total population!!AGE!!Under 5 years,Estimate!!Total!!Total population!!AGE!!5 to 9 years,Margin of Error!!Total!!Total population!!AGE!!5 to 9 years,Estimate!!Total!!Total population!!AGE!!10 to 14 years,Margin of Error!!Total!!Total population!!AGE!!10 to 14 years,Estimate!!Total!!Total population!!AGE!!15 to 19 years,Margin of Error!!Total!!Total population!!AGE!!15 to 19 years,Estimate!!Total!!Total population!!AGE!!20 to 24 years,Margin of Error!!Total!!Total population!!AGE!!20 to 24 years,Estimate!!Total!!Total population!!AGE!!25 to 29 years,Margin of Error!!Total!!Total population!!AGE!!25 to 29 years,Estimate!!Total!!Total population!!AGE!!30 to 34 years,Margin of Error!!Total!!Total population!!AGE!!30 to 34 years,Estimate!!Total!!Total population!!AGE!!35 to 39 years,Margin of Error!!Total!!Total population!!AGE!!35 to 39 years,Estimate!!Total!!Total population!!AGE!!40 to 44 years,Margin of Error!!Total!!Total population!!AGE!!40 to 44 years,Estimate!!Total!!Total population!!AGE!!45 to 49 years,Margin of Error!!Total!!Total population!!AGE!!45 to 49 years,Estimate!!Total!!Total population!!AGE!!50 to 54 years,Margin of Error!!Total!!Total population!!AGE!!50 to 54 years,Estimate!!Total!!Total population!!AGE!!55 to 59 years,Margin of Error!!Total!!Total population!!AGE!!55 to 59 years,Estimate!!Total!!Total population!!AGE!!60 to 64 years,Margin of Error!!Total!!Total population!!AGE!!60 to 64 years,Estimate!!Total!!Total population!!AGE!!65 to 69 years,Margin of Error!!Total!!Total population!!AGE!!65 to 69 years,Estimate!!Total!!Total population!!AGE!!70 to 74 years,Margin of Error!!Total!!Total population!!AGE!!70 to 74 years,Estimate!!Total!!Total population!!AGE!!75 to 79 years,Margin of Error!!Total!!Total population!!AGE!!75 to 79 years,Estimate!!Total!!Total population!!AGE!!80 to 84 years,Margin of Error!!Total!!Total population!!AGE!!80 to 84 years,Estimate!!Total!!Total population!!AGE!!85 years and over,Margin of Error!!Total!!Total population!!AGE!!85 years and over,Estimate!!Total!!Total population!!SELECTED AGE CATEGORIES!!5 to 14 years,Margin of Error!!Total!!Total population!!SELECTED AGE CATEGORIES!!5 to 14 years,Estimate!!Total!!Total population!!SELECTED AGE CATEGORIES!!15 to 17 years,Margin of Error!!Total!!Total population!!SELECTED AGE CATEGORIES!!15 to 17 years,Estimate!!Total!!Total population!!SELECTED AGE CATEGORIES!!Under 18 years,Margin of Error!!Total!!Total population!!SELECTED AGE CATEGORIES!!Under 18 years,Estimate!!Total!!Total population!!SELECTED AGE CATEGORIES!!18 to 24 years,Margin of Error!!Total!!Total population!!SELECTED AGE CATEGORIES!!18 to 24 years,Estimate!!Total!!Total population!!SELECTED AGE CATEGORIES!!15 to 44 years,Margin of Error!!Total!!Total population!!SELECTED AGE CATEGORIES!!15 to 44 years,Estimate!!Total!!Total population!!SELECTED AGE CATEGORIES!!16 years and over,Margin of Error!!Total!!Total population!!SELECTED AGE CATEGORIES!!16 years and over,Estimate!!Total!!Total population!!SELECTED AGE CATEGORIES!!18 years and over,Margin of Error!!Total!!Total population!!SELECTED AGE CATEGORIES!!18 years and over,Estimate!!Total!!Total population!!SELECTED AGE CATEGORIES!!21 years and over,Margin of Error!!Total!!Total population!!SELECTED AGE CATEGORIES!!21 years and over,Estimate!!Total!!Total population!!SELECTED AGE CATEGORIES!!60 years and over,Margin of Error!!Total!!Total population!!SELECTED AGE CATEGORIES!!60 years and over,Estimate!!Total!!Total population!!SELECTED AGE CATEGORIES!!62 years and over,Margin of Error!!Total!!Total population!!SELECTED AGE CATEGORIES!!62 years and over,Estimate!!Total!!Total population!!SELECTED AGE CATEGORIES!!65 years and over,Margin of Error!!Total!!Total population!!SELECTED AGE CATEGORIES!!65 years and over,Estimate!!Total!!Total population

In [13]:
census_age['clean_name'] = census_age['Geographic Area Name'].apply(clean_name)

In [14]:
census_age = census_age[['id', 'clean_name', 'Estimate!!Total!!Total population', 
                        'Estimate!!Total!!Total population!!AGE!!15 to 19 years', 'Estimate!!Total!!Total population!!AGE!!20 to 24 years',
                       'Estimate!!Total!!Total population!!AGE!!25 to 29 years','Estimate!!Total!!Total population!!AGE!!30 to 34 years']]

In [15]:
#clean up the names to make reading the data easier
census_age.rename({'Estimate!!Total!!Total population':'total_pop_estimate', 
                   'Estimate!!Total!!Total population!!AGE!!15 to 19 years':'15_to_19',
                  'Estimate!!Total!!Total population!!AGE!!20 to 24 years':'20_to_24', 
                   'Estimate!!Total!!Total population!!AGE!!25 to 29 years':'25_to_29', 
                  'Estimate!!Total!!Total population!!AGE!!30 to 34 years':'30_to_34'}, axis=1, inplace=True)
census_age.head()

,id,clean_name,total_pop_estimate,15_to_19,20_to_24,25_to_29,30_to_34
1,310M500US10140,"Aberdeen, WA",75061,4828,3184,4062,4529
2,310M500US10180,"Abilene, TX",171795,13195,15460,12915,11620
3,310M500US10300,"Adrian, MI",98451,6044,6519,5415,5464
4,310M500US10380,"Aguadilla-Isabela, PR",288877,17627,21061,15736,17630
5,310M500US10420,"Akron, OH",703479,45160,52153,47606,43989


In [16]:
#change strings to ints
for col in ['total_pop_estimate','15_to_19','20_to_24','25_to_29','30_to_34']:
    census_age[col] = census_age[col].astype(int)

In [17]:
#get the percentage of the population that is in each age group
def get_pop_percentage(pop_estimate, subgroup):
    percentage = subgroup/pop_estimate
    return percentage*100

for col in ['15_to_19', '20_to_24', '25_to_29', '30_to_34']:
    census_age[f'{col}_percentage'] = census_age.apply(lambda x: get_pop_percentage(x.total_pop_estimate, x[col]), axis=1)
    
    
    
    
    

In [18]:
#add together 'young' age groups and get that proportion of the population
census_age['total_young_pop'] = census_age['15_to_19'] + census_age['20_to_24']+census_age['25_to_29']+census_age['30_to_34']
census_age['young_pop_percent'] = (census_age.total_young_pop / census_age.total_pop_estimate) *100

In [19]:
census_age.head(3)

,id,clean_name,total_pop_estimate,15_to_19,20_to_24,25_to_29,30_to_34,15_to_19_percentage,20_to_24_percentage,25_to_29_percentage,30_to_34_percentage,total_young_pop,young_pop_percent
1,310M500US10140,"Aberdeen, WA",75061,4828,3184,4062,4529,6.432102,4.241883,5.411599,6.033759,16603,22.119343
2,310M500US10180,"Abilene, TX",171795,13195,15460,12915,11620,7.680666,8.999098,7.517681,6.763876,53190,30.961320
3,310M500US10300,"Adrian, MI",98451,6044,6519,5415,5464,6.139095,6.621568,5.500198,5.549969,23442,23.810830


In [20]:
census_age = census_age[census_age.total_young_pop<88000000]

In [ ]:
#save cleaned data to csv
census_age.to_csv('Cleaned_Census_Data.csv')

## College Data

Another area we could potentially target is colleges & universities, which are full of young people who often need to travel home. Below we clean data on US colleges & universities for mapping. In the end, it will include info on the school, its lat/long, tuition data, and numbers of out of state students that could be relevant advertising targets. 

In [21]:
school_df = pd.read_csv('Base_Ed_Spreadsheet.csv')

In [22]:
pd.options.display.max_columns=100
school_df.drop(labels=['number_enrolled_pt', 'open_admissions_policy', 'inst_control', 'institution_level',
                      'inst_category'], axis=1, inplace=True)
school_df.head(3)

,year,unitid,inst_name,state_name,number_enrolled_ft,hbcu,tribal_college,date_closed,inst_status,open_public,postsec_public_active,inst_size,cc_undergrad_2018,cc_size_setting_2018,dist_progs_all,calendar_system
0,2018,100654,Alabama A & M University,Alabama,1525.0,Yes,No,Not applicable,Active,Yes,Active postsecondary institution,"5,000-9,999","Four-year, nearly all full-time, inclusive acc...","Four-year, medium, highly residential",No,Semester
1,2018,100663,University of Alabama at Birmingham,Alabama,2245.0,No,No,Not applicable,Active,Yes,Active postsecondary institution,"20,000 and above","Four-year, mostly full-time, selective accepta...","Four-year, large, primarily nonresidential",No,Semester
2,2018,100690,Amridge University,Alabama,NaN,No,No,Not applicable,Active,Yes,Active postsecondary institution,"Under 1,000","Four-year, mostly part-time","Four-year, very small, primarily nonresidential",Yes,Semester


In [23]:
#filter any schools out that aren't residential or that are very small
school_df = school_df[~school_df.cc_undergrad_2018.str.contains('mostly part-time')&
                     ~school_df.cc_undergrad_2018.str.contains('Not classified')&
                     ~school_df.cc_undergrad_2018.str.contains('mixed part/full-time')&
                     ~school_df.cc_undergrad_2018.str.contains('not accredited')&
                     ~school_df.cc_size_setting_2018.str.contains('primarily nonresidential')&
                     ~school_df.cc_size_setting_2018.str.contains('very small')]

#only include schools that are in the states we're targeting based on airline data
school_df = school_df[(school_df.state_name == 'Pennsylvania') |(school_df.state_name == 'Massachusetts')| (school_df.state_name == 'New York')|
                     (school_df.state_name == 'District of Columbia')| (school_df.state_name=='California')| (school_df.state_name=='North Carolina')|
                     (school_df.state_name=='Colorado')|(school_df.state_name=='Utah')|(school_df.state_name=='Oregon')|(school_df.state_name=='Illinois')|
                     (school_df.state_name=='Florida')|(school_df.state_name=='Washington') | (school_df.state_name=='New Jersey')]

#must be active schools
school_df = school_df[school_df.postsec_public_active == "Active postsecondary institution"]

#can't be only distance education schools
school_df = school_df[school_df.dist_progs_all == "No"]

In [24]:
school_df.drop(labels=['year', 'date_closed', 'inst_status', 'open_public', 'postsec_public_active', 
                      'dist_progs_all', 'calendar_system'], axis=1, inplace=True)
school_df.shape

(462, 9)

In [25]:
#get the tuition fees for each school 
tuition = pd.read_csv('School_Tuition_Breakdown.csv')

In [26]:
tuition = tuition[(tuition.level_of_study == 'Undergraduate') & (tuition.tuition_type!='In district')]
tuition.sort_values(by=['unitid', 'tuition_type'], inplace=True)

In [27]:
tuition.head(5)

,year,unitid,inst_name,state_name,level_of_study,tuition_type,tuition_fees_ft
1,2018,100654,Alabama A & M University,Alabama,Undergraduate,In state,9744.0
2,2018,100654,Alabama A & M University,Alabama,Undergraduate,Out of state,18354.0
7,2018,100663,University of Alabama at Birmingham,Alabama,Undergraduate,In state,8568.0
8,2018,100663,University of Alabama at Birmingham,Alabama,Undergraduate,Out of state,19704.0
13,2018,100690,Amridge University,Alabama,Undergraduate,In state,9900.0


In [28]:
tuition.dropna(inplace=True)
#find the difference between each school's instate and out of state tuition
diffs = np.diff(tuition.tuition_fees_ft)

In [29]:
#insert nan at the beginning of the diffs array to realign it
diffs2 = np.insert(diffs, 0, np.nan)
diffs2

array([   nan,  8610., -9786., ...,     0., -5604.,     0.])

In [30]:
#add in the diffs array as a column
tuition['outofstate-instate'] = diffs2

In [31]:
#filter out the in state rows
tuition = tuition[tuition.tuition_type=="Out of state"]

In [32]:
tuition.head(3)

,year,unitid,inst_name,state_name,level_of_study,tuition_type,tuition_fees_ft,outofstate-instate
2,2018,100654,Alabama A & M University,Alabama,Undergraduate,Out of state,18354.0,8610.0
8,2018,100663,University of Alabama at Birmingham,Alabama,Undergraduate,Out of state,19704.0,11136.0
14,2018,100690,Amridge University,Alabama,Undergraduate,Out of state,9900.0,0.0


In [33]:
#merge college and tuition datasets
school_tuition = school_df.merge(tuition, on='unitid', how='left')

In [34]:
school_tuition.head(3)

,unitid,inst_name_x,state_name_x,number_enrolled_ft,hbcu,tribal_college,inst_size,cc_undergrad_2018,cc_size_setting_2018,year,inst_name_y,state_name_y,level_of_study,tuition_type,tuition_fees_ft,outofstate-instate
0,109785,Azusa Pacific University,California,1107.0,No,No,"10,000-19,999","Four-year, nearly all full-time, selective acc...","Four-year, medium, highly residential",2018.0,Azusa Pacific University,California,Undergraduate,Out of state,38880.0,0.0
1,110097,Biola University,California,857.0,No,No,"5,000-9,999","Four-year, nearly all full-time, selective acc...","Four-year, medium, highly residential",2018.0,Biola University,California,Undergraduate,Out of state,40488.0,0.0
2,110361,California Baptist University,California,1570.0,No,No,"10,000-19,999","Four-year, nearly all full-time, selective acc...","Four-year, medium, primarily residential",2018.0,California Baptist University,California,Undergraduate,Out of state,33478.0,0.0


In [35]:
#this dataset contains the lat/long data for the schools
geographies = pd.read_excel('School_Geographies.xlsx')

In [36]:
#if the school ID is in our school_tuition dataset, include it
geographies= geographies[geographies.UNITID.isin(school_tuition.unitid.unique())]

In [37]:
geographies.drop(labels=['STREET', 'STFIP', 'CNTY', 'NMCNTY', 'LOCALE', 'CBSATYPE', 'CSA', 'NMCSA', 'NECTA', 'NMNECTA',
                        'CD', 'SLDL', 'SLDU', 'SCHOOLYEAR'], axis=1, inplace=True)

In [38]:
geographies.rename({'UNITID':'unitid'}, axis=1, inplace=True)
geographies.head(3)


,unitid,NAME,CITY,STATE,ZIP,LAT,LON,CBSA,NMCBSA
213,109785,Azusa Pacific University,Azusa,CA,91702-7000,34.130047,-117.888375,31080,"Los Angeles-Long Beach-Anaheim, CA"
218,110097,Biola University,La Mirada,CA,90639-0001,33.906203,-118.014374,31080,"Los Angeles-Long Beach-Anaheim, CA"
224,110361,California Baptist University,Riverside,CA,92504-3297,33.929321,-117.425619,40140,"Riverside-San Bernardino-Ontario, CA"


In [39]:
#merge school & tuition dataset with the latitude and longitude
school_tuition_latlong = school_tuition.merge(geographies, on='unitid', how='left')

In [40]:
school_tuition_latlong.head(3)

,unitid,inst_name_x,state_name_x,number_enrolled_ft,hbcu,tribal_college,inst_size,cc_undergrad_2018,cc_size_setting_2018,year,inst_name_y,state_name_y,level_of_study,tuition_type,tuition_fees_ft,outofstate-instate,NAME,CITY,STATE,ZIP,LAT,LON,CBSA,NMCBSA
0,109785,Azusa Pacific University,California,1107.0,No,No,"10,000-19,999","Four-year, nearly all full-time, selective acc...","Four-year, medium, highly residential",2018.0,Azusa Pacific University,California,Undergraduate,Out of state,38880.0,0.0,Azusa Pacific University,Azusa,CA,91702-7000,34.130047,-117.888375,31080,"Los Angeles-Long Beach-Anaheim, CA"
1,110097,Biola University,California,857.0,No,No,"5,000-9,999","Four-year, nearly all full-time, selective acc...","Four-year, medium, highly residential",2018.0,Biola University,California,Undergraduate,Out of state,40488.0,0.0,Biola University,La Mirada,CA,90639-0001,33.906203,-118.014374,31080,"Los Angeles-Long Beach-Anaheim, CA"
2,110361,California Baptist University,California,1570.0,No,No,"10,000-19,999","Four-year, nearly all full-time, selective acc...","Four-year, medium, primarily residential",2018.0,California Baptist University,California,Undergraduate,Out of state,33478.0,0.0,California Baptist University,Riverside,CA,92504-3297,33.929321,-117.425619,40140,"Riverside-San Bernardino-Ontario, CA"


In [41]:
#clean up a few inconsistent metro area names
def clean_CBSA(row):
    if row == "Raleigh-Cary, NC":
        return "Raleigh, NC"
    elif row == "Sacramento-Roseville-Folsom, CA":
        return "Sacramento--Roseville--Arden-Arcade, CA"
    else:
        return row
    
school_tuition_latlong['NMCBSA'] = school_tuition_latlong['NMCBSA'].apply(clean_CBSA)

In [42]:
#this dataset contains info on how many freshmen students were
#from out of state
out_of_staters = pd.read_csv('Out_of_Staters.csv')

#only include rows that include schools in our geographies as well
#as students who are from the state at the other end of the Amtrak route
def pick_rows(state_name, res_state):
    if state_name == "Massachusetts":
        if res_state in ["Pennsylvania", 'New Jersey', 'District of Columbia']:
            return 1
    elif state_name == "California":
        if res_state in ["California", "Oregon"]:
            return 1
    elif state_name == "New York":
        if res_state in ["North Carolina"]:
            return 1
    elif state_name == "Colorado":
        if res_state in ["Utah", "Illinois"]:
            return 1
    elif state_name == "Oregon":
        if res_state in ["California"]:
            return 1
    elif state_name == "Pennsylvania":
        if res_state in ["Massachusetts"]:
            return 1
    elif state_name == "New Jersey":
        if res_state in ["Massachusetts"]:
            return 1
    elif state_name == "District of Columbia":
        if res_state in ["Massachusetts"]:
            return 1
    elif state_name == "North Carolina":
        if res_state in ["New York"]:
            return 1
    elif state_name == "Utah":
        if res_state in ["Colorado"]:
            return 1
    elif state_name =="Illinois":
        if res_state in ["Colorado"]:
            return 1
        
        
out_of_staters['include'] = out_of_staters.apply(lambda x: pick_rows(x.state_name, x.state_of_residence), axis=1)

In [43]:
out_of_staters = out_of_staters[out_of_staters.include==1]

In [44]:
out_of_staters.head(3)

,year,unitid,inst_name,state_name,state_of_residence,type_of_freshman,enrollment_fall,include
3335,2018,108232,Academy of Art University,California,California,Total,203.0,1.0
3338,2018,108232,Academy of Art University,California,Oregon,Total,7.0,1.0
3347,2018,108232,Academy of Art University,California,Oregon,Graduated from HS in past 12 months,5.0,1.0


In [45]:
#merge the out of state data into our dataset
merged_schools_freshmen = school_tuition_latlong.merge(out_of_staters, on="unitid", how="left")

In [46]:
merged_schools_freshmen.head(3)

,unitid,inst_name_x,state_name_x,number_enrolled_ft,hbcu,tribal_college,inst_size,cc_undergrad_2018,cc_size_setting_2018,year_x,inst_name_y,state_name_y,level_of_study,tuition_type,tuition_fees_ft,outofstate-instate,NAME,CITY,STATE,ZIP,LAT,LON,CBSA,NMCBSA,year_y,inst_name,state_name,state_of_residence,type_of_freshman,enrollment_fall,include
0,109785,Azusa Pacific University,California,1107.0,No,No,"10,000-19,999","Four-year, nearly all full-time, selective acc...","Four-year, medium, highly residential",2018.0,Azusa Pacific University,California,Undergraduate,Out of state,38880.0,0.0,Azusa Pacific University,Azusa,CA,91702-7000,34.130047,-117.888375,31080,"Los Angeles-Long Beach-Anaheim, CA",2018.0,Azusa Pacific University,California,California,Total,786.0,1.0
1,109785,Azusa Pacific University,California,1107.0,No,No,"10,000-19,999","Four-year, nearly all full-time, selective acc...","Four-year, medium, highly residential",2018.0,Azusa Pacific University,California,Undergraduate,Out of state,38880.0,0.0,Azusa Pacific University,Azusa,CA,91702-7000,34.130047,-117.888375,31080,"Los Angeles-Long Beach-Anaheim, CA",2018.0,Azusa Pacific University,California,California,Graduated from HS in past 12 months,776.0,1.0
2,109785,Azusa Pacific University,California,1107.0,No,No,"10,000-19,999","Four-year, nearly all full-time, selective acc...","Four-year, medium, highly residential",2018.0,Azusa Pacific University,California,Undergraduate,Out of state,38880.0,0.0,Azusa Pacific University,Azusa,CA,91702-7000,34.130047,-117.888375,31080,"Los Angeles-Long Beach-Anaheim, CA",2018.0,Azusa Pacific University,California,Oregon,Total,23.0,1.0


In [47]:
#filter down to only the geographies that are relevant in our Amtrak analysis
merged_schools_freshmen=merged_schools_freshmen[(merged_schools_freshmen.NMCBSA == "Boston-Cambridge-Newton, MA-NH")|
                                               (merged_schools_freshmen.NMCBSA == "Los Angeles-Long Beach-Anaheim, CA")|
                                               (merged_schools_freshmen.NMCBSA == "New York-Newark-Jersey City, NY-NJ-PA")|
                                               (merged_schools_freshmen.NMCBSA == "Denver-Aurora-Lakewood, CO")|
                                               (merged_schools_freshmen.NMCBSA == "Portland-Vancouver-Hillsboro, OR-WA")|
                                               (merged_schools_freshmen.NMCBSA == "Philadelphia-Camden-Wilmington, PA-NJ-DE-MD")|
                                               (merged_schools_freshmen.NMCBSA == "Washington-Arlington-Alexandria, DC-VA-MD-WV")|
                                               (merged_schools_freshmen.NMCBSA == "San Jose-Sunnyvale-Santa Clara, CA")|
                                               (merged_schools_freshmen.NMCBSA == "Raleigh, NC")|
                                               (merged_schools_freshmen.NMCBSA == "Sacramento--Roseville--Arden-Arcade, CA")|
                                               (merged_schools_freshmen.NMCBSA == "Salt Lake City, UT")|
                                               (merged_schools_freshmen.NMCBSA == "Chicago-Naperville-Elgin, IL-IN-WI")]

In [ ]:
#save all of the cleaned data for a backup
school_tuition_latlong.to_csv('Schools_All_Info.csv')
out_of_staters.to_csv('out_of_staters_clean.csv')
merged_schools_freshmen.to_csv('merged_schools_and_makeup.csv')